<a href="https://colab.research.google.com/github/peteray-dev/AML_Car_Price_Prediction/blob/master/Sentiment_Analysis_of_Financial_News.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q cohere praw newsapi-python pandas transformers peft torch huggingface_hub pytz datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 116.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install tweepy


In [3]:
from datetime import datetime, timedelta
import praw
import cohere
from newsapi import NewsApiClient
import pandas as pd
import logging
import pytz
import time
import warnings
from transformers import LlamaForCausalLM, LlamaTokenizerFast
from peft import PeftModel
import torch
from collections import defaultdict
import numpy as np
import random
from huggingface_hub import login
import tweepy

warnings.filterwarnings("ignore")

In [4]:
from google.colab import userdata

reddit_client_id = userdata.get('REDDIT_CLIENT_ID')
reddit_client_secret = userdata.get('REDDIT_CLIENT_SECRET')
reddit_user_agent = userdata.get('REDDIT_USER_AGENT')
newsapi_key = userdata.get('NEWSAPI_KEY')
cohere_api_key = userdata.get('COHERE_KEY')
huggingface_api_key = userdata.get('HF_TOKEN')
twitter_bearer_token = userdata.get('TWITTER_BEARER_TOKEN')
tickers = ["AAPL", "TSLA"]

In [5]:
login(huggingface_api_key)

In [6]:
model = LlamaForCausalLM.from_pretrained("FinGPT/fingpt-mt_llama3-8b_lora")
tokenizer = LlamaTokenizerFast.from_pretrained("meta-llama/Meta-Llama-3-8B")

adapter_config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

adapter_model.safetensors:   0%|          | 0.00/13.6M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'PreTrainedTokenizerFast'. 
The class this function is called from is 'LlamaTokenizerFast'.
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


In [7]:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})  # Add the padding token


1

In [17]:
class TextFetchPipeline:
  def __init__(self, newsapi_key, reddit_client_id, reddit_client_secret, reddit_user_agent, cohere_api_key, tickers, model, tokenizer, twitter_bearer_token):
    self.news_api = NewsApiClient(api_key=newsapi_key)
    self.reddit = praw.Reddit(client_id = reddit_client_id, client_secret = reddit_client_secret, user_agent = reddit_user_agent)
    self.co = cohere.Client(cohere_api_key)
    self.twitter_client = tweepy.Client(bearer_token=twitter_bearer_token, wait_on_rate_limit=True)
    self.model = model
    self.tokenizer = tokenizer
    self.news_cache = set()
    self.reddit_cache = set()
    self.agg_text = {}
    self.sentiment = defaultdict()
    self.tickers = defaultdict()
    self.prop = defaultdict()

    self.ticker_company_map = {"AAPL": "Apple", "TSLA": "Tesla"}  # Add more as needed

    for ticker in tickers:
      self.tickers[ticker] = self.ticker_company_map[ticker]
      self.sentiment[ticker] = 'Neutral'
      self.prop[ticker] = 1.0

  def fetch_news(self, ticker):
    company_name = self.tickers[ticker]
    # print(company_name)
    now = datetime.utcnow()
    thirty_minutes_ago = now - timedelta(days=2) #2 days ago
    now_str = now.strftime('%Y-%m-%dT%H:%M:%S')
    thirty_minutes_ago_str = thirty_minutes_ago.strftime('%Y-%m-%dT%H:%M:%S')
    print(ticker)
    articles = self.news_api.get_everything(q=f"{ticker} OR {company_name}", from_param=thirty_minutes_ago_str, to=now_str, language='en', sort_by='publishedAt' ) #relevancy
    # print(articles)
    new_articles = []
    print(f"NewsAPI query: {ticker} | From: {thirty_minutes_ago_str} | To: {now_str}")
    for article in articles.get("articles", []):
      title = article['title']
      published_at = datetime.strptime(article["publishedAt"], '%Y-%m-%dT%H:%M:%SZ')
      if title not in self.news_cache and published_at > thirty_minutes_ago:
        self.news_cache.add(title)
        new_articles.append({"source": "news", "text":title, "ticker": ticker})
    print(f"Number of new articles: {new_articles}")
    return new_articles

  def fetch_reddit(self, ticker):
    company_name = self.tickers[ticker]
    reddit_posts = []
    current_time_utc = datetime.utcnow().replace(tzinfo=pytz.utc)
    self.reddit_cache = {(post_id, ts) for post_id, ts in self.reddit_cache if ts > current_time_utc - timedelta(days=10)}

    for post in self.reddit.subreddit('wallstreetbets').new(limit=50):
        text = post.selftext.lower()
        # print(text)
        post_time_utc = datetime.utcfromtimestamp(post.created_utc).replace(tzinfo=pytz.UTC)

        if (post.id, post_time_utc) not in self.reddit_cache and (ticker.lower() in text or company_name.lower() in text.lower()):
            reddit_posts.append({"source": "reddit", "text": text, "ticker": ticker})
            self.reddit_cache.add((post.id, post_time_utc))

    return reddit_posts


  # def fetch_twitter(self, ticker):
  #   company_name = self.tickers[ticker]
  #   query = f'("{ticker}" OR "{company_name}") -is:retweet lang:en'

  #   try:
  #       tweets = self.twitter_client.search_recent_tweets(
  #           query=query,
  #           max_results=10,
  #           tweet_fields=["created_at", "text"]
  #       )
  #       tweet_data = []
  #       if tweets.data:
  #           for tweet in tweets.data:
  #               if tweet.text not in self.news_cache:
  #                   self.news_cache.add(tweet.text)
  #                   tweet_data.append({
  #                       "source": "twitter",
  #                       "text": tweet.text,
  #                       "ticker": ticker
  #                   })

  #       # Respect rate limits
  #       time.sleep(10)  # pause 3s to avoid 429s when looping multiple tickers
  #       return tweet_data
  #   except Exception as e:
  #       print(f"Twitter fetch error: {e}")
  #       return []

  def fetch_combined_data(self):
    combined_data = []
    for ticker in self.tickers.keys():
        news_data = self.fetch_news(ticker)
        reddit_data = self.fetch_reddit(ticker)
        # twitter_data = self.fetch_twitter(ticker)
        ticker_data = news_data + reddit_data #+ twitter_data
        combined_data.extend(ticker_data)
        print(f"Combined data for {ticker}: {ticker_data}")  # only print data for current ticker
        # time.sleep(10)  # pause 3s to avoid 429s when looping multiple tickers
    return combined_data

  def summarize_text(self, text):
    response = self.co.summarize(text = text, length = "short", format="paragraph", model="summarize-xlarge")
    return response.summary

  def process_combined_data_with_summary(self):
    try:
      est = pytz.timezone('Europe/London')
      raw_data = self.fetch_combined_data()
      if not raw_data:
        for ticker in self.tickers:
          self.agg_text[ticker] = "No data available"
        return

      for entry in raw_data:
        entry["time"] = datetime.utcnow().replace(tzinfo=pytz.utc).astimezone(est).strftime('%Y-%m-%d %H:%M:%S')
      df = pd.DataFrame(raw_data)
      # print(df)
      aggregated_df = df.groupby("ticker").agg({'time': "first", "text": lambda texts: " ".join(texts)}).reset_index()
      # print(aggregated_df)
      aggregated_df["summary"] = aggregated_df["text"].apply(self.summarize_text)
      for ticker in self.tickers:
        filtered_df = aggregated_df[aggregated_df["ticker"] == ticker]
        self.agg_text[ticker] = filtered_df["summary"].values[0] if not filtered_df.empty else ""
    except Exception as e:
      logging.error(f"Error in process_combined_data_with_summary: {str(e)}")



  def get_sentiment(self, text):
    if not text:
      return "Neutral", 1.0
    text = text[:2000]
    prompt = f'''Instruction: What is the sentiment of this news? Please choose an answer from [Positive, Negative, Neutral].\nInput: {text}\nAnswer: '''
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    inputs = self.tokenizer(prompt, return_tensors="pt", padding=True, max_length=128).to(device)
    with torch.no_grad():
      outputs = self.model(**inputs)
    logits = outputs.logits[:, -1, :]
    probs = torch.softmax(logits, dim=-1)
    class_tokens = self.tokenizer(["Positive", "Negative", "Neutral"], add_special_tokens = False)["input_ids"]
    class_probs = {self.tokenizer.decode(token_id): probs[0, token_id].item() for token_id in class_tokens}
    sentiment = max(class_probs, key=class_probs.get)
    prob = np.round(class_probs[sentiment], 3)
    return sentiment, prob



In [18]:
pipeline = TextFetchPipeline(newsapi_key, reddit_client_id, reddit_client_secret, reddit_user_agent, cohere_api_key, tickers, model, tokenizer, twitter_bearer_token)

# Step 1: Generate and store summaries
pipeline.process_combined_data_with_summary()

# Step 2: Run sentiment on each summary
# for ticker, summary_text in pipeline.agg_text.items():
#     sentiment, prob = pipeline.get_sentiment(summary_text)
#     print(f"{ticker}: Sentiment = {sentiment} ({prob})")

AAPL


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



NewsAPI query: AAPL | From: 2025-04-29T18:12:45 | To: 2025-05-01T18:12:45
Number of new articles: [{'source': 'news', 'text': 'iPhone 17 Series’ Dummies Appear in Hands-On Video, 17 Air Thiness is Exposed', 'ticker': 'AAPL'}, {'source': 'news', 'text': 'The Great Gardening of 2025 – Part 15 – Laurel Leaves', 'ticker': 'AAPL'}, {'source': 'news', 'text': 'Apple notifies new victims of spyware attacks across the world', 'ticker': 'AAPL'}, {'source': 'news', 'text': 'For All Mankind’s ‘Race for the Base’ could soon become reality', 'ticker': 'AAPL'}, {'source': 'news', 'text': "In court, CEO Sundar Pichai defends Google against the DOJ's 'extraordinary' proposals", 'ticker': 'AAPL'}, {'source': 'news', 'text': 'The Queue: Another day in the login queue', 'ticker': 'AAPL'}, {'source': 'news', 'text': "PlayStation Plus Subscribers Can Play One of 2024's Best Games in May", 'ticker': 'AAPL'}, {'source': 'news', 'text': 'This Gag From Apple TV+ Show ‘The Studio’ Turned Into Reality And We’re 

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



NewsAPI query: TSLA | From: 2025-04-29T18:12:47 | To: 2025-05-01T18:12:47
Number of new articles: [{'source': 'news', 'text': 'What’s Actually on the Table in Trump’s Promised Trade Deals?', 'ticker': 'TSLA'}, {'source': 'news', 'text': 'Elon Musk bids farewell — sort of — to Trump Cabinet, claims DOGE saved $160 billion', 'ticker': 'TSLA'}, {'source': 'news', 'text': 'Save up to $5,394 on Anker’s SOLIX F3800 Plus offers, Jackery takes up to 43% off units, RadExpand 5 e-bike at one-day $1,099 low, more', 'ticker': 'TSLA'}, {'source': 'news', 'text': 'SCOOP: DOGE Caucus plans White House meeting as Elon Musk preps to step back', 'ticker': 'TSLA'}, {'source': 'news', 'text': 'First Slate Auto and now Isuzu – why electric pick-up trucks could be the next big EV battleground', 'ticker': 'TSLA'}, {'source': 'news', 'text': 'Alijah Arenas: Teen Son of Gilbert Arenas Out of Coma, Released From Hospital', 'ticker': 'TSLA'}, {'source': 'news', 'text': 'Waymo & Toyota May Collaborate On Autonomo

In [19]:
# Step 2: Run sentiment on each summary
for ticker, summary_text in pipeline.agg_text.items():
    sentiment, prob = pipeline.get_sentiment(summary_text)
    print(f"{ticker}: Sentiment = {sentiment} ({prob})")

AAPL: Sentiment = Positive (0.516)
TSLA: Sentiment = Neutral (0.782)
